# JWST Data Query and Download from MAST

This notebook provides a complete workflow for:
1. Querying MAST for JWST observations at specific sky coordinates
2. Displaying available data products (instruments, filters, exposure times)
3. Interactively selecting and downloading data products at various processing stages

**Requirements:** `astroquery`, `astropy`, `pandas`, `numpy`

## Setup: Import Required Libraries

In [1]:
# Core astronomy libraries
from astroquery.mast import Observations, Mast
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.table import Table
import astropy.io.fits as fits

# Data manipulation and display
import pandas as pd
import numpy as np

# System utilities
import os
from pathlib import Path
import warnings
from IPython.display import display, HTML
from datetime import datetime

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

print("✓ All libraries imported successfully")

✓ All libraries imported successfully


## Part 1: Query MAST by Coordinates

Enter your target coordinates below. The default coordinates are for SMACS 0723 (one of the first JWST deep field targets).

In [2]:
# ===== USER INPUT: SET YOUR TARGET COORDINATES =====


# Define the sexagesimal coordinates
ra_h = 12
ra_m = 11
ra_s = 59.31

dec_deg = 11
dec_m = 48
dec_s = 46.80

# Convert RA from hours to degrees (1 hour = 15 degrees)
ra_decimal = (ra_h + ra_m / 60 + ra_s / 3600) * 15

# Convert Dec to decimal degrees
dec_decimal = dec_deg + dec_m / 60 + dec_s / 3600

target_ra  = ra_decimal
target_dec = dec_decimal

search_radius = 2.0      # Search radius in arcminutes

# Alternative examples (uncomment to use):
# CEERS field: target_ra, target_dec = 214.825, 52.825
# GOODS-S: target_ra, target_dec = 53.1, -27.8
# UNCOVER (Abell 2744): target_ra, target_dec = 3.587, -30.398

print(f"Target Coordinates:")
print(f"  RA:  {target_ra}°")
print(f"  Dec: {target_dec}°")
print(f"  Search Radius: {search_radius} arcmin")

Target Coordinates:
  RA:  182.997125°
  Dec: 11.813°
  Search Radius: 2.0 arcmin


In [3]:
# Create SkyCoord object for the target
target_coord = SkyCoord(ra=target_ra*u.degree, dec=target_dec*u.degree, frame='icrs')

print(f"\nQuerying MAST for JWST observations...")
print(f"Coordinates: {target_coord.to_string('hmsdms')}")

# Query MAST for JWST observations with retry logic
import time

max_retries = 3
retry_delay = 5  # seconds

for attempt in range(max_retries):
    try:
        print(f"Attempt {attempt + 1}/{max_retries}...")
        
        obs_table = Observations.query_criteria(
            coordinates=target_coord,
            radius=search_radius*u.arcmin,
            obs_collection='JWST',
            dataproduct_type='image'
        )
        
        if len(obs_table) == 0:
            print("\n⚠ No JWST observations found at these coordinates.")
            print("Try increasing the search radius or checking different coordinates.")
        else:
            print(f"\n✓ Found {len(obs_table)} JWST observations!")
        
        break  # Success, exit retry loop
        
    except Exception as e:
        error_msg = str(e)
        
        if "Logon failed" in error_msg or "trigger execution" in error_msg:
            print(f"\n⚠ MAST server issue detected (Attempt {attempt + 1}/{max_retries})")
            print("   This is a temporary server-side problem at STScI.")
            
            if attempt < max_retries - 1:
                print(f"   Retrying in {retry_delay} seconds...")
                time.sleep(retry_delay)
                retry_delay *= 2  # Exponential backoff
            else:
                print("\n✗ MAST service is currently unavailable.")
                obs_table = None
        else:
            print(f"\n✗ Error querying MAST: {e}")
            obs_table = None
            break



Querying MAST for JWST observations...
Coordinates: 12h11m59.31s +11d48m46.8s
Attempt 1/3...

✓ Found 18 JWST observations!


## Filter for NIRCam, NIRISS, and MIRI Instruments

In [4]:
if obs_table is not None and len(obs_table) > 0:
    # Filter for the instruments of interest
    instruments_of_interest = ['NIRCAM', 'NIRISS', 'MIRI', 'NIRSPEC']
    
    # Get unique instruments in the observations
    all_instruments = set(obs_table['instrument_name'])
    print(f"All instruments found in query: {all_instruments}\n")
    
    print("\n" + "="*60)
    print("INSTRUMENT SUMMARY")
    print("="*60)
    
    # Use substring matching since instrument names include mode (e.g., 'NIRCAM/IMAGE')
    filtered_indices = []
    
    for inst in instruments_of_interest:
        # Find observations where instrument_name contains the instrument string
        inst_mask = np.array([inst in str(name) for name in obs_table['instrument_name']])
        count = np.sum(inst_mask)
        
        if count > 0:
            print(f"✓ {inst:8s}: {count:3d} observation(s) found")
            filtered_indices.extend(np.where(inst_mask)[0])
        else:
            print(f"✗ {inst:8s}: No observations found")
    
    # Filter table to only include instruments of interest
    if len(filtered_indices) > 0:
        # Remove duplicates and sort
        filtered_indices = sorted(set(filtered_indices))
        filtered_obs = obs_table[filtered_indices]
        print(f"\nTotal relevant observations: {len(filtered_obs)}")
    else:
        filtered_obs = None
        print("\nNo observations match the selected instruments.")
else:
    filtered_obs = None
    print("No observations to filter.")

All instruments found in query: {np.str_('NIRCAM/IMAGE'), np.str_('NIRSPEC/IFU')}


INSTRUMENT SUMMARY
✓ NIRCAM  :   2 observation(s) found
✗ NIRISS  : No observations found
✗ MIRI    : No observations found
✓ NIRSPEC :  16 observation(s) found

Total relevant observations: 18


## Part 2: Display Available Data Details

Show detailed information about filters, exposure times, and observation metadata.

In [5]:
def create_observation_summary(obs_table):
    """
    Create a detailed summary DataFrame of observations.
    """
    if obs_table is None or len(obs_table) == 0:
        return None
    
    # Extract relevant columns
    summary_data = {
        'Obs ID': obs_table['obs_id'],
        'Instrument': obs_table['instrument_name'],
        'Filters': obs_table['filters'],
        'Exposure (s)': obs_table['t_exptime'],
        'Proposal ID': obs_table['proposal_id'],
        'Target Name': obs_table['target_name'],
        'Date': [str(d).split('T')[0] if d else 'N/A' for d in obs_table['t_min']],
    }
    
    df = pd.DataFrame(summary_data)
    return df

if filtered_obs is not None and len(filtered_obs) > 0:
    summary_df = create_observation_summary(filtered_obs)
    
    print("\n" + "="*60)
    print("DETAILED OBSERVATION SUMMARY")
    print("="*60 + "\n")
    
    # Get unique instrument base names (e.g., 'NIRCAM' from 'NIRCAM/IMAGE')
    unique_instruments = set()
    for inst_name in summary_df['Instrument']:
        # Extract base instrument name (before '/')
        base_inst = str(inst_name).split('/')[0]
        unique_instruments.add(base_inst)
    
    # Display by instrument (only those actually present)
    for inst in sorted(unique_instruments):
        # Use substring matching to find all observations with this instrument
        inst_data = summary_df[summary_df['Instrument'].str.contains(inst, na=False)]
        
        if len(inst_data) > 0:
            print(f"\n{'='*60}")
            print(f"{inst} OBSERVATIONS ({len(inst_data)} total)")
            print(f"{'='*60}")
            
            # Display table
            display(inst_data.style.set_properties(**{
                'text-align': 'left',
                'font-size': '11pt'
            }))
            
            # Show unique filters for this instrument
            unique_filters = set()
            for f in inst_data['Filters']:
                if f and str(f) != 'nan':
                    # Split multiple filters if present
                    filters_list = str(f).split(';')
                    unique_filters.update(filters_list)
            
            if unique_filters:
                print(f"\n  Unique Filters: {', '.join(sorted(unique_filters))}")
            
            # Show exposure time statistics
            exp_times = inst_data['Exposure (s)'].dropna()
            if len(exp_times) > 0:
                print(f"  Exposure Time Range: {exp_times.min():.1f} - {exp_times.max():.1f} seconds")
                print(f"  Total Exposure Time: {exp_times.sum():.1f} seconds ({exp_times.sum()/3600:.2f} hours)")
else:
    summary_df = None
    print("No observations to summarize.")


DETAILED OBSERVATION SUMMARY


NIRCAM OBSERVATIONS (2 total)


,Obs ID,Instrument,Filters,Exposure (s),Proposal ID,Target Name,Date
0,jw06779-o001_t001_nircam_clear-f200w,NIRCAM/IMAGE,F200W,7558.688000,6779,FRBLocation,60670.709302152776
1,jw06779-o001_t001_nircam_clear-f322w2,NIRCAM/IMAGE,F322W2,7558.688000,6779,FRBLocation,60670.709302152776



  Unique Filters: F200W, F322W2
  Exposure Time Range: 7558.7 - 7558.7 seconds
  Total Exposure Time: 15117.4 seconds (4.20 hours)

NIRSPEC OBSERVATIONS (16 total)


,Obs ID,Instrument,Filters,Exposure (s),Proposal ID,Target Name,Date
2,jw06779002001_12101_00004_nrs2,NIRSPEC/IFU,CLEAR;PRISM,1458.889000,6779,FRBLocation-NIRSpec-Pointing,60697.449281099536
3,jw06779002001_08101_00002_nrs2,NIRSPEC/IFU,CLEAR;PRISM,1458.889000,6779,FRBLocation-NIRSpec-Pointing,60697.26776275463
4,jw06779002001_06101_00003_nrs2,NIRSPEC/IFU,CLEAR;PRISM,1458.889000,6779,FRBLocation-NIRSpec-Pointing,60697.212547256946
5,jw06779002001_06101_00002_nrs2,NIRSPEC/IFU,CLEAR;PRISM,1458.889000,6779,FRBLocation-NIRSpec-Pointing,60697.19464875
6,jw06779002001_10101_00002_nrs2,NIRSPEC/IFU,CLEAR;PRISM,1458.889000,6779,FRBLocation-NIRSpec-Pointing,60697.34053898148
7,jw06779002001_06101_00004_nrs2,NIRSPEC/IFU,CLEAR;PRISM,1458.889000,6779,FRBLocation-NIRSpec-Pointing,60697.2306146412
8,jw06779002001_06101_00001_nrs2,NIRSPEC/IFU,CLEAR;PRISM,1458.889000,6779,FRBLocation-NIRSpec-Pointing,60697.17675025463
9,jw06779002001_08101_00004_nrs2,NIRSPEC/IFU,CLEAR;PRISM,1458.889000,6779,FRBLocation-NIRSpec-Pointing,60697.30355975695
10,jw06779002001_12101_00002_nrs2,NIRSPEC/IFU,CLEAR;PRISM,1458.889000,6779,FRBLocation-NIRSpec-Pointing,60697.413484097226
11,jw06779002001_08101_00003_nrs2,NIRSPEC/IFU,CLEAR;PRISM,1458.889000,6779,FRBLocation-NIRSpec-Pointing,60697.28566126157



  Unique Filters: CLEAR, PRISM
  Exposure Time Range: 1458.9 - 1458.9 seconds
  Total Exposure Time: 23342.2 seconds (6.48 hours)


## Part 3: Data Product Selection and Download

### Step 3.1: Get Available Data Products



In [6]:
# Let's limit for NIRCam imaging data for now
instrument = 'NIRCAM/IMAGE'
filtered_obs = filtered_obs[filtered_obs['instrument_name'] == instrument]

In [7]:
def get_data_products(obs_table):
    """
    Retrieve all data products for the observations.
    """
    if obs_table is None or len(obs_table) == 0:
        return None
    
    print("Retrieving data products from MAST...")
    print("(This may take a moment for large numbers of observations)\n")
    
    try:
        data_products = Observations.get_product_list(obs_table)
        print(f"✓ Found {len(data_products)} total data products")
        return data_products
    except Exception as e:
        print(f"✗ Error retrieving data products: {e}")
        return None

if filtered_obs is not None and len(filtered_obs) > 0:
    data_products = get_data_products(filtered_obs)
else:
    data_products = None

Retrieving data products from MAST...
(This may take a moment for large numbers of observations)

✓ Found 4412 total data products


### Step 3.2: Analyze Available Processing Stages

In [9]:
def analyze_available_stages(data_products):
    """
    Analyze which calibration stages are available for each observation.
    """
    if data_products is None or len(data_products) == 0:
        return None
    
    # Map calibration levels to stage names
    stage_mapping = {
        '0': 'Stage 0 (Uncalibrated)',
        '1': 'Stage 1 (Detector-level)',
        '2': 'Stage 2 (Calibrated)',
        '3': 'Stage 3 (Combined)'
    }
    
    # Filter for FITS files (science data)
    science_products = data_products[
        (data_products['productType'] == 'SCIENCE') 
    ]
    
    # Analyze by observation ID
    obs_stage_info = {}

    for obs_id in np.unique(science_products['obs_id']):
        obs_products = science_products[science_products['obs_id'] == obs_id]
        
        # Check which stages are available based on filename patterns
        available_stages = set()
        stage_counts = {}
        
        for product in obs_products:
            
    
            
            filename = product['productFilename']
            
            # Identify stage from filename
            if '_uncal.fits' in filename:
                stage = '0'
            elif '_rate.fits' in filename or '_rateints.fits' in filename:
                stage = '1'
            elif '_cal.fits' in filename or '_calints.fits' in filename:
                stage = '2'
            elif '_i2d.fits' in filename or '_x1d.fits' in filename or '_c1d.fits' in filename:
                stage = '3'
            else:
                continue
            
            available_stages.add(stage)
            stage_counts[stage] = stage_counts.get(stage, 0) + 1
        
        obs_stage_info[obs_id] = {
            'stages': sorted(available_stages),
            'counts': stage_counts,
            'total_products': len(obs_products)
        }
    
    return obs_stage_info, science_products

if data_products is not None and len(data_products) > 0:
    stage_info, science_products = analyze_available_stages(data_products)
    
    if stage_info:
        print("\n" + "="*60)
        print("AVAILABLE CALIBRATION STAGES BY OBSERVATION")
        print("="*60 + "\n")
        
        stage_mapping = {
            '0': 'Stage 0 (Uncalibrated)',
            '1': 'Stage 1 (Detector-level)',
            '2': 'Stage 2 (Calibrated)',
            '3': 'Stage 3 (Combined)'
        }
        
        for i, (obs_id, info) in enumerate(stage_info.items(), 1):
            print(f"{i}. Observation: {obs_id}")
            print(f"   Total Products: {info['total_products']}")
            print(f"   Available Stages:")
            for stage in info['stages']:
                count = info['counts'][stage]
                print(f"     ✓ {stage_mapping[stage]}: {count} file(s)")
            print()
else:
    stage_info = None
    science_products = None


AVAILABLE CALIBRATION STAGES BY OBSERVATION

1. Observation: jw06779-o001_t001_nircam_clear-f200w
   Total Products: 2
   Available Stages:
     ✓ Stage 3 (Combined): 1 file(s)

2. Observation: jw06779-o001_t001_nircam_clear-f322w2
   Total Products: 2
   Available Stages:
     ✓ Stage 3 (Combined): 1 file(s)

3. Observation: jw06779001001_02101_00001_nrca1
   Total Products: 6
   Available Stages:
     ✓ Stage 0 (Uncalibrated): 1 file(s)
     ✓ Stage 1 (Detector-level): 2 file(s)
     ✓ Stage 2 (Calibrated): 1 file(s)
     ✓ Stage 3 (Combined): 1 file(s)

4. Observation: jw06779001001_02101_00001_nrca2
   Total Products: 6
   Available Stages:
     ✓ Stage 0 (Uncalibrated): 1 file(s)
     ✓ Stage 1 (Detector-level): 2 file(s)
     ✓ Stage 2 (Calibrated): 1 file(s)
     ✓ Stage 3 (Combined): 1 file(s)

5. Observation: jw06779001001_02101_00001_nrca3
   Total Products: 6
   Available Stages:
     ✓ Stage 0 (Uncalibrated): 1 file(s)
     ✓ Stage 1 (Detector-level): 2 file(s)
     ✓ Stag

### Step 3.3: Interactive Selection and Download

**Instructions:**
1. Choose which observations to download (by index number)
2. Select the calibration stage(s) you want
3. Specify the download directory
4. Run the download cell

In [11]:
# ===== USER INPUT: SELECT OBSERVATIONS AND STAGES TO DOWNLOAD =====

# Select observation indices (from the list above)
# Example: [1, 2, 3] to download first three observations
# Use [0] to download ALL observations
selected_obs_indices = [0]  # Change this to your desired observation numbers

# Select calibration stages to download
# Options: '0', '1', '2', '3' or ['0', '1', '2', '3'] for all
selected_stages = ['3']  # Stage 2 (calibrated) and Stage 3 (combined)

# Download directory
download_dir = 'MAST_Downloads/'  # Change to your preferred location

print("Selection Summary:")
print(f"  Observations: {selected_obs_indices if selected_obs_indices != [0] else 'ALL'}")
print(f"  Stages: {', '.join(selected_stages)}")
print(f"  Download Directory: {download_dir}")

Selection Summary:
  Observations: ALL
  Stages: 3
  Download Directory: MAST_Downloads/


In [12]:
def download_selected_products(stage_info, science_products, obs_indices, stages, output_dir):
    """
    Download selected data products with progress tracking.
    """
    if stage_info is None or science_products is None:
        print("No data products available to download.")
        return
    
    # Create output directory
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    
    # Get list of observation IDs
    obs_ids = list(stage_info.keys())
    
    # Determine which observations to download
    if obs_indices == [0]:
        selected_obs_ids = obs_ids
    else:
        selected_obs_ids = [obs_ids[i-1] for i in obs_indices if 0 < i <= len(obs_ids)]
    
    print(f"\nPreparing to download data for {len(selected_obs_ids)} observation(s)...\n")
    
    # Map stage numbers to calibration levels
    stage_to_calib_level = {
        '0': 1,  # Stage 0 (Uncalibrated) -> calib_level 1
        '1': 2,  # Stage 1 (Detector-level) -> calib_level 2
        '2': 2,  # Stage 2 (Calibrated) -> calib_level 2
        '3': 3   # Stage 3 (Combined) -> calib_level 3
    }
    
    # Collect products to download
    products_to_download = []
    
    for obs_id in selected_obs_ids:
        obs_products = science_products[science_products['obs_id'] == obs_id]
        
        for stage in stages:
            if stage not in stage_info[obs_id]['stages']:
                print(f"⚠ Warning: Stage {stage} not available for {obs_id}, skipping...")
                continue
            
            calib_level = stage_to_calib_level.get(stage)
            
            # Filter by calibration level
            stage_products = obs_products[obs_products['calib_level'] == calib_level]
            
            for product in stage_products:
                products_to_download.append(product)
    
    if len(products_to_download) == 0:
        print("No products match your selection criteria.")
        return
    
    print(f"Total files to download: {len(products_to_download)}")
    
    # Calculate approximate download size
    total_size_mb = sum([p['size'] for p in products_to_download if p['size']]) / (1024*1024)
    print(f"Approximate total size: {total_size_mb:.2f} MB\n")
    
    # Confirm download
    response = input("Proceed with download? (yes/no): ").strip().lower()
    
    if response not in ['yes', 'y']:
        print("Download cancelled.")
        return
    
    print(f"\nDownloading to: {output_dir}\n")
    print("="*60)
    
    # Convert to astropy Table for download
    download_table = Table(products_to_download)
    
    try:
        # Download with progress tracking
        manifest = Observations.download_products(
            download_table,
            download_dir=output_dir
        )
        
        print("="*60)
        print(f"\n✓ Download complete!")
        print(f"  Total files downloaded: {len(manifest)}")
        print(f"  Location: {output_dir}")
        
        # Show summary of what was downloaded
        print("\nDownload Summary by Calibration Level:")
        stage_names = {
            '0': 'Stage 0 (Uncalibrated, calib_level=1)',
            '1': 'Stage 1 (Detector-level, calib_level=2)',
            '2': 'Stage 2 (Calibrated, calib_level=2)',
            '3': 'Stage 3 (Combined, calib_level=3)'
        }
        
        for stage in stages:
            calib_level = stage_to_calib_level[stage]
            count = sum([1 for p in products_to_download if p['calib_level'] == calib_level])
            print(f"  {stage_names[stage]}: {count} files")
        
        return manifest
        
    except Exception as e:
        print(f"\n✗ Error during download: {e}")
        return None

# Execute download
if stage_info is not None:
    download_manifest = download_selected_products(
        stage_info,
        science_products,
        selected_obs_indices,
        selected_stages,
        download_dir
    )
else:
    print("No observations available for download. Please run the query cells first.")


Preparing to download data for 82 observation(s)...

Total files to download: 4
Approximate total size: 1909.47 MB

Proceed with download? (yes/no): yes



✓ Download complete!
  Total files downloaded: 4
  Location: MAST_Downloads/

Download Summary by Calibration Level:
  Stage 3 (Combined, calib_level=3): 4 files


## Additional Utilities

### View Downloaded Files

In [13]:
def list_downloaded_files(download_dir):
    """
    List all FITS files in the download directory.
    """
    download_path = Path(download_dir)
    
    if not download_path.exists():
        print(f"Directory {download_dir} does not exist.")
        return
    
    fits_files = sorted(download_path.rglob('*.fits'))
    
    if len(fits_files) == 0:
        print(f"No FITS files found in {download_dir}")
        return
    
    print(f"\nFound {len(fits_files)} FITS files in {download_dir}:\n")
    
    # Group by directory
    from collections import defaultdict
    files_by_dir = defaultdict(list)
    
    for f in fits_files:
        files_by_dir[f.parent].append(f.name)
    
    for directory, files in sorted(files_by_dir.items()):
        rel_dir = directory.relative_to(download_path)
        print(f"\n{rel_dir}/")
        for fname in sorted(files):
            file_size = (directory / fname).stat().st_size / (1024*1024)
            print(f"  - {fname} ({file_size:.2f} MB)")

# List downloaded files
if download_dir:
    list_downloaded_files(download_dir)


Found 2 FITS files in MAST_Downloads/:


mastDownload/JWST/jw06779-o001_t001_nircam_clear-f200w/
  - jw06779-o001_t001_nircam_clear-f200w_i2d.fits (1574.51 MB)

mastDownload/JWST/jw06779-o001_t001_nircam_clear-f322w2/
  - jw06779-o001_t001_nircam_clear-f322w2_i2d.fits (328.06 MB)


### Quick FITS Header Check

In [14]:
def inspect_fits_header(filepath, keywords=None):
    """
    Display key header information from a FITS file.
    """
    if keywords is None:
        # Default keywords to display
        keywords = [
            'INSTRUME', 'FILTER', 'PUPIL', 'DETECTOR', 
            'EXPSTART', 'EXPTIME', 'EFFEXPTM',
            'TARG_RA', 'TARG_DEC', 'PROGRAM'
        ]
    
    try:
        with fits.open(filepath) as hdul:
            header = hdul[0].header
            
            print(f"\nFITS Header Info: {Path(filepath).name}")
            print("="*60)
            
            for key in keywords:
                if key in header:
                    print(f"{key:12s}: {header[key]}")
                    
    except Exception as e:
        print(f"Error reading {filepath}: {e}")

# Example: inspect the first downloaded file (if any)
if download_dir:
    fits_files = sorted(Path(download_dir).rglob('*.fits'))
    if len(fits_files) > 0:
        print("\nInspecting first downloaded file:")
        inspect_fits_header(fits_files[0])
    else:
        print("\nNo FITS files available to inspect.")


Inspecting first downloaded file:

FITS Header Info: jw06779-o001_t001_nircam_clear-f200w_i2d.fits
INSTRUME    : NIRCAM
FILTER      : F200W
PUPIL       : CLEAR
DETECTOR    : MULTIPLE
EXPSTART    : 60670.709302152776
EFFEXPTM    : 7558.688000000001
TARG_RA     : 182.99720833333333
TARG_DEC    : 11.812997222222222
PROGRAM     : 06779


## Summary and Next Steps

This notebook has provided:
1. ✓ Coordinate-based MAST queries for JWST data
2. ✓ Detailed observation summaries (instruments, filters, exposure times)
3. ✓ Interactive selection and download of data products at various stages

**Next Steps:**
- Use a FITS viewer (e.g., DS9, QFitsView) to inspect downloaded files

**Useful Resources:**
- [MAST JWST Documentation](https://outerspace.stsci.edu/display/MASTDOCS/JWST+Advanced+Search+Tutorials)
- [JWST Calibration Pipeline](https://jwst-pipeline.readthedocs.io/)
- [astroquery.mast Documentation](https://astroquery.readthedocs.io/en/latest/mast/mast.html)